In [ ]:
!pip install pydub
!pip install audiomentations

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.3/82.3 kB 1.0 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import os
import shutil
import random
from pydub import AudioSegment
from audiomentations import Compose, AddGaussianNoise, TimeStretch, PitchShift
import numpy as np

In [ ]:
input_folder = "/content/drive/MyDrive/mlcourse_finalresult/0602_15000val_audio_sample"
output_folder = "/content/drive/MyDrive/mlcourse_finalresult/0602_15000_audio_sample_augmented"


if not os.path.exists(output_folder):
    os.makedirs(output_folder)

audio_files = [file for file in os.listdir(input_folder) if file.endswith(".wav")]



前5个音频文件:
['노인남여_노인대화71_F_milkgirl999_65_경상_실내_77817.wav', '노인남여_노인대화64_F_samjeon814_61_수도권_실내_70835.wav', '노인남여_노인대화46_F_jihun1495_63_수도권_실내_51485.wav', '노인남여_노인대화12_M_YSD00_68_경상_녹음실_15277.wav', '노인남여_노인대화07_M_1527864598_70_강원_실내_08320.wav']
总共有15000个音频文件


In [ ]:
audio_files = os.listdir(input_folder)

random.shuffle(audio_files)

num_files = len(audio_files)
num_files_per_part = num_files // 4

In [ ]:
# data augmentation
augment_noise = Compose([
    AddGaussianNoise(min_amplitude=0.0001, max_amplitude=0.001, p=1.0)
])
augment_reverb = Compose([
    TimeStretch(min_rate=0.95, max_rate=1.05, p=1.0)
])
augment_distortion = Compose([
    PitchShift(min_semitones=-1, max_semitones=1, p=1.0)
])

In [ ]:

def apply_augmentation(augmentation, input_path, output_path):
    audio = AudioSegment.from_file(input_path)
    samples = audio.get_array_of_samples()
    sample_rate = audio.frame_rate

    samples = np.array(samples).astype(np.float32) / 32768.0
    if augmentation is not None:
        augmented_samples = augmentation(samples=samples, sample_rate=sample_rate)
    else:
        augmented_samples = samples
    augmented_samples = (augmented_samples * 32768.0).astype(np.int16)

    augmented_audio = AudioSegment(
        data=augmented_samples.tobytes(),
        sample_width=audio.sample_width,
        frame_rate=audio.frame_rate,
        channels=audio.channels
    )

    augmented_audio.export(output_path, format="wav")


for i, audio_file in enumerate(audio_files):

    src_path = os.path.join(input_folder, audio_file)
    dst_path = os.path.join(output_folder, audio_file)

    if i < num_files_per_part:
        apply_augmentation(None, src_path, dst_path)
    elif i < num_files_per_part * 2:
        apply_augmentation(augment_noise, src_path, dst_path)
    elif i < num_files_per_part * 3:
        apply_augmentation(augment_reverb, src_path, dst_path)
    else:
        apply_augmentation(augment_distortion, src_path, dst_path)

print("sucess!")

增强完成!
